In [2]:
import os
from subprocess import PIPE, run
import time
import pandas as pd
import numpy as np 
import pprint
import requests


#### Data Bullshit

In [3]:
Data = pd.read_csv('/Users/sergiomares/Desktop/Nunez/Jin file/TSS_CpG_crispriphenotype_table.txt', delimiter = '\t',header = 0)
Data = Data.sort_values(["gene",'average phenotype of strongest 3'], ascending = True).drop_duplicates(subset = 'gene', keep = 'first')
UCSC_TSS = pd.read_csv('UCSC_TSS.txt', delimiter = '\t',header = 0)
UCSC_TSS = UCSC_TSS.drop_duplicates() 

errors = []

for gene in (Data['gene']):
        try:
                if len(UCSC_TSS[UCSC_TSS['hg19.kgXref.geneSymbol'] == gene]["hg19.knownGene.txStart"]) > 1:
                        x = abs(UCSC_TSS[UCSC_TSS['hg19.kgXref.geneSymbol'] == gene]["hg19.knownGene.txStart"] - Data[Data['gene'] == gene]["Primary TSS, 3'"].iloc[-1]).min()
                        Data.loc[Data['gene'] == gene, "Primary TSS, 3'"] = (int(Data[Data['gene'] == gene]["Primary TSS, 3'"].iloc[-1] - int(x) + 1))
                else:
                        x = UCSC_TSS[UCSC_TSS['hg19.kgXref.geneSymbol'] == gene]["hg19.knownGene.txStart"].iloc[-1]
                        Data.loc[Data['gene'] == gene, "Primary TSS, 3'"] = int(x) + 1 
              

        except:
            errors.append(gene)

len(errors)

Data = Data.reset_index()

In [4]:
Promoter_sequences = pd.read_csv('3kb_Promoter.sequences-2.csv', sep=',', header = 0)
Promoter_sequences = pd.merge(how = 'outer', left = Promoter_sequences, right = Data, left_on = 'Gene', right_on = 'gene')
Promoter_sequences = Promoter_sequences.dropna()

In [5]:
cpgs = pd.read_csv('1-s2-S0092867421003536-mmc3.csv',sep = ',',  header = 0)
cpgs = pd.DataFrame(cpgs)

In [6]:
df = pd.merge(how = 'outer', left = Promoter_sequences, right = cpgs, left_on = 'Gene', right_on = 'gene')
df = df.loc[df.Gene.notna()]
df = df.reset_index()

In [7]:
irbs = pd.read_csv('wgEncodeHaibMethylRrbsK562HaibSitesRep1.bed', sep='\t', header = 0)
#Extract only the values with 100% Certainty of methylation 
irbs = irbs[irbs['Unnamed: 8'] == '255,0,0']

chromosome_list = np.unique(irbs.track)

In [8]:
tmp = np.zeros((len(df),3001))

for index, chromosomes in enumerate(chromosome_list):
    
    table = irbs[irbs['track'] == chromosomes]
    table2 = df[df.chromosome == chromosomes]

    for i, x in enumerate(table['name="SL725.1']):


        for o, z in enumerate(table2["Primary TSS, 3'"]):

            if (x > (z - 1500) and (x < (z + 1500))) == True:
                y = x - (z - 1500)
                #print((z - 1500), z,  (z + 1500))
                #print('CpG Islands found in:', x, 'Position on vector:', y,"in row", Promoter_sequences['gene'][o], o + 1)    

                tmp[df.level_0[df["Primary TSS, 3'"] == z ].iloc[-1] + 1][int(y)] = 1

In [9]:
## Function to show each position for a certain gene

import matplotlib.pyplot as plt

#### Nilah recommended creating a plot for methylation near TSS (position and its phenotype score)

def methylated_basepairs(int):
    if len(np.where(tmp[int] == 1)[0]) > 0:
        is_c(int)
        is_c(int)

        x = df.head(int).tail(1)['Gene'].iloc[-1]
        # print(x, "| Coordinates of TSS:", df[df['Gene'] == x]["Primary TSS, 3'"].iloc[-1])
        # print("Found", len([np.where(tmp[int] == 1)][0][0]) , "Methylated basepairs at positions:")
        # print([np.where(tmp[int] == 1)][0][0] + df[df['Gene'] == x]["Primary TSS, 3'"].iloc[-1] - 1500)
        # plt.title("Methylation profile for promoter region of gene: {}".format(x), size = 15)
        # plt.xlabel('Position of the bases | 1500 = Start of TSS ')
        # plt.ylabel('Methylation')
        # plt.plot(tmp[int])

        # #print(df[df['Gene'] == x]["chromosome"].iloc[-1],':',[np.where(tmp[int] == 1)][0][0][0] + df[df['Gene'] == x]["Primary TSS, 3'"].iloc[-1] - 1500, '-', [np.where(tmp[int] == 1)][0][0][0] + df[df['Gene'] == x]["Primary TSS, 3'"].iloc[-1] - 1500 + 1)
        # print(df[df['Gene'] == x]["chromosome"].iloc[-1],':', df[df['Gene'] == x]["Primary TSS, 3'"].iloc[-1] - 1500, '-', df[df['Gene'] == x]["Primary TSS, 3'"].iloc[-1] + 1500 )

        #return([np.where(tmp[int] == 1)][0][0])

        return(len([np.where(tmp[int] == 1)][0][0] + df[df['Gene'] == x]["Primary TSS, 3'"].iloc[-1]))

    else:
        return 0

# Are the predicted methylation sites C?  

def is_c(int):
    
    counter = 0
    total = 0

    try:
        x = [np.where(tmp[int] == 1)][0][0]
        y = df.head(int).tail(1)['Gene'].iloc[-1]

        if len(x) == 0:
            #print("No Methylated Cs")
            counter = 2
            
        for i in x:

            total += 1

            if list(df.Sequences[df['Gene'] == y].iloc[-1])[i] == 'C':
                #print(i, 'is a C')
                counter +=1 
            else:
                #print(i, 'is not a C, its a ' + list(df.Sequences[df['Gene'] == y].iloc[-1])[i])
                tmp[int][i] = 0
    except:
        print("Issue with int", int, "Gene:")

    #print('Correct Cs:',counter/total*100,'%')



In [10]:
methylated_cs = [methylated_basepairs(i) for i in range(20340)]
methylated_cs

[0,
 0,
 1,
 0,
 0,
 0,
 13,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 9,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 5,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 5,
 30,
 8,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 14,
 0,
 0,
 0,
 0,
 4,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 14,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 7,
 7,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 4,
 3,
 0,
 2,
 0,
 3,
 0,
 5,
 0,
 0,
 7,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 3,
 5,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,

In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

sequences = df.Sequences

 # This removes empty sequences.

# The LabelEncoder encodes a sequence of bases as a sequence of integers.
integer_encoder = LabelEncoder()  
# The OneHotEncoder converts an array of integers to a sparse matrix where 
# each row corresponds to one possible value of each feature.
one_hot_encoder = OneHotEncoder(categories='auto')   
input_features = []

counter = 0 
for sequence in sequences:
    try:
        integer_encoded = integer_encoder.fit_transform(list(sequence))
        integer_encoded = np.array(integer_encoded).reshape(-1, 1)
        one_hot_encoded = one_hot_encoder.fit_transform(integer_encoded)
        input_features.append(one_hot_encoded.toarray())
    except:
        print('error with', sequence)

indexes = []

print(len(input_features))

for i in range(len(input_features)):
    try:
        if input_features[i].shape != (3001, 4):
            indexes.append(i)
            input_features.pop(i)
            print(i)
    except:
        print('error with', i)    
print(len(input_features))

20340
2079
3491
4014
5695
5971
6588
7741
9579
9831
10451
11164
19212
error with 20328
error with 20329
error with 20330
error with 20331
error with 20332
error with 20333
error with 20334
error with 20335
error with 20336
error with 20337
error with 20338
error with 20339
20328


In [12]:
np.set_printoptions(threshold=40)
input_features = np.stack(input_features)
print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n',sequences[0][:10],'...',sequences[0][-10:])
print('One hot encoding of Sequence #1:\n',input_features[0].T)

Example sequence
-----------------------
DNA Sequence #1:
 CAAGGAAAAC ... AGCCAAGATT
One hot encoding of Sequence #1:
 [[0. 1. 1. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]]


In [13]:
print(len(tmp))
tmp_1 = np.delete(tmp, indexes, axis = 0)
print(len(tmp_1))

20340
20328


In [14]:
tmp_1.shape

(20328, 3001)

In [15]:
md_input_features = np.dstack((input_features, tmp_1))

In [16]:
print(len(df['CRISPRoff_average']))
df = df.drop(index = indexes)
print(len(df['CRISPRoff_average']))

20340
20328


In [17]:
# from tensorflow.keras import backend as K
# K.clear_session()

In [18]:
from sklearn.model_selection import train_test_split

train_features_1, test_features, train_labels, test_labels = train_test_split(
    input_features, df['CRISPRoff_average'], test_size=0.25, random_state=42)

## Pytorch

In [42]:
import torch
import torchvision
from torchvision import transforms, datasets

In [228]:

input_features = np.stack(md_input_features)
print("Example sequence\n-----------------------")
print('DNA Sequence #1:\n',sequences[0][:10],'...',sequences[0][-10:])
print('Classes: ', integer_encoder.classes_)
print('One hot encoding of Sequence #1:\n',input_features[0].T)
input_features.shap

input_labels = df['CRISPRoff_average']

print('Labels:\n',labels.T)
print('One-hot encoded labels:\n',input_labels.T)


# %% Split data
input_features = input_features.transpose(0,2,1).astype(np.float32) # adapt to pytorch input format [N, C, L]
input_features.shape
input_labels = input_labels.astype(np.float32) # adapt to pytorch input format
input_labels.shape
train_features, test_features, train_labels, test_labels = train_test_split(input_features, input_labels, test_size=0.25, random_state=42)
train_features, val_features, train_labels, val_labels =  train_test_split(train_features, train_labels, test_size=0.25, random_state=42)


Example sequence
-----------------------
DNA Sequence #1:
 CCGAGGGCTA ... CGCGGACACC
Classes:  ['A' 'C' 'G' 'T']
One hot encoding of Sequence #1:
 [[0. 1. 1. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
Labels:
 [['0' '0' '0' ... '0' '1' '1']]
One-hot encoded labels:
 0       -0.063267
1        0.029424
2        0.021009
3        0.012236
4        0.006076
           ...   
20335    0.020289
20336    0.012916
20337    0.004499
20338    0.029630
20339   -0.242959
Name: CRISPRoff_average, Length: 20328, dtype: float64


In [204]:
train_loader = torch.utils.data.DataLoader(list(zip(train_features, train_labels)), batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(list(zip(val_features, val_labels)), batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(list(zip(test_features, test_labels)), batch_size=1, shuffle=False)

In [205]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(4, 32, 12)
        self.pool = nn.MaxPool1d(4)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(288, 16)
        self.fc2 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)

        return x

In [206]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

In [217]:
# Hyper parameters
num_epochs = 50
num_classes = 1
batch_size = 100
learning_rate = 0.001

In [208]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [218]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/50], Step [100/282], Loss: 0.0024
Epoch [1/50], Step [200/282], Loss: 0.0038
Epoch [2/50], Step [100/282], Loss: 0.0031
Epoch [2/50], Step [200/282], Loss: 0.0084
Epoch [3/50], Step [100/282], Loss: 0.0005
Epoch [3/50], Step [200/282], Loss: 0.0017
Epoch [4/50], Step [100/282], Loss: 0.0006
Epoch [4/50], Step [200/282], Loss: 0.0002
Epoch [5/50], Step [100/282], Loss: 0.0001
Epoch [5/50], Step [200/282], Loss: 0.0000
Epoch [6/50], Step [100/282], Loss: 0.0002
Epoch [6/50], Step [200/282], Loss: 0.0001
Epoch [7/50], Step [100/282], Loss: 0.0003
Epoch [7/50], Step [200/282], Loss: 0.0002
Epoch [8/50], Step [100/282], Loss: 0.0009
Epoch [8/50], Step [200/282], Loss: 0.0006
Epoch [9/50], Step [100/282], Loss: 0.0001
Epoch [9/50], Step [200/282], Loss: 0.0003
Epoch [10/50], Step [100/282], Loss: 0.0001
Epoch [10/50], Step [200/282], Loss: 0.0002
Epoch [11/50], Step [100/282], Loss: 0.0004
Epoch [11/50], Step [200/282], Loss: 0.0004
Epoch [12/50], Step [100/282], Loss: 0.0000
Epoch 

In [219]:
model.eval()

Net(
  (conv1): Conv1d(4, 32, kernel_size=(12,), stride=(1,))
  (pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=288, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=2, bias=True)
)

In [220]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 100.0 %


In [221]:
torch.save(model.state_dict(), 'model.ckpt')

## Old Tensorflow model

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [17]:
from tensorflow.keras.layers import Dense, MaxPooling1D, Conv1D, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

model_in = Input(shape=(train_features_1[1].shape))
model = (Conv1D(filters = 15, kernel_size = 5))(model_in)

model1 = (Conv1D(filters = 3, kernel_size = 2))(model)
# model1 = (MaxPooling1D(pool_size=(7)))(model1)
# model1 =(Dense(14, activation='relu'))(model1)
# model1 = (MaxPooling1D(pool_size=(7)))(model1)
model1 = (Dense(1, activation='relu'))(model1)

model = Model(model_in, model1)
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3001, 4)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 2997, 15)          315       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2996, 3)           93        
_________________________________________________________________
dense (Dense)                (None, 2996, 1)           4         
Total params: 412
Trainable params: 412
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print('done')

In [259]:
#hyp = [29, 19, 2, 39, 15, 3]
hyp = [15, 5, 15, 2, 7, 14]

model_to_test = run_model(build_model(hyp))

print(average_percentage_error(model_to_test))

# plt.title('first: f ='+str(hyp[0])+'k ='+ str(hyp[1]) + '\n second: f ='+ str(hyp[2]) + 'k =' + str(hyp[3]) +
#          '\n pool size =' + str(hyp[4]) + '\n Dense layer =' + str(hyp[5]))
# plt.scatter(test_labels['mutant_average'], test_labels['CRISPRoff_average'], c='black',s = 0.5)
# plt.scatter(predicted[0], predicted[1], c='red',s = 0.1)
# plt.figure()
# plt.show()

In [251]:
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=12, 
                 input_shape=(train_features.shape[1], 4)))
model.add(MaxPooling1D(pool_size=4))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['binary_accuracy'])
model.summary()

ModuleNotFoundError: No module named 'keras'

In [258]:
import tensorflow as tf

def build_model(list):
    model_in = Input(shape=(train_features[1].shape))
    model = (Conv1D(filters = list[0], kernel_size = list[1], padding = 'same'))(model_in)

    model1 = (Conv1D(filters = list[2], kernel_size = list[3], padding = 'same'))(model)
    model1 = (MaxPooling1D(pool_size=(list[4])))(model1)
    model1 =(Dense(list[5], activation='relu'))(model1)
    model1 = GlobalMaxPool1D()(model1)
    model1 = (Dense(1, activation='relu'))(model1)

    model2 = (Conv1D(filters = list[2], kernel_size = list[3], padding = 'same'))(model)
    model2 = (MaxPooling1D(pool_size=(list[4])))(model2)
    model2 =(Dense(list[5], activation='relu'))(model2)
    model2 = GlobalMaxPool1D()(model2)
    model2 = (Dense(1, activation='relu'))(model2)

    model = Model(model_in, [model1,model2])
    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

    return model

def run_model(model):
    tf.random.set_seed(2)
    model.fit(train_features, train_labels, batch_size=50, verbose=0, validation_split=0.35)
    predicted = model.predict(test_features)

    return(predicted)


def average_percentage_error(predicted):

    predicted_values[0], predicted_values[1] = pd.DataFrame(predicted[0]), pd.DataFrame(predicted[1])

    values = (0, 0)

    for i in range(4674):
        temp = abs(test_labels[i] - predicted_values[0][i])/test_labels[i] * 100
        values = values[0] + temp
    
    values =  values[0]/4674, values[1]/4674

    return values


def good_parameter(evaluation, previous_evaluation, parameter_list, index_of_parameter, best_list):

    if evaluation[0] < previous_evaluation[0] and evaluation[1] < previous_evaluation[1]:
        best_list = parameter_list
        if index_of_parameter < 5:
            index_of_parameter = index_of_parameter + 1
        else:
            index_of_parameter = 0 
        parameter_list[index_of_parameter] = parameter_list[index_of_parameter] + 1
        return(parameter_list, index_of_parameter, evaluation, best_list)
    else:
        parameter_list[index_of_parameter] = parameter_list[index_of_parameter] - 1
        if index_of_parameter < 5:
            index_of_parameter = index_of_parameter + 1
        else:
            index_of_parameter = 0 
        parameter_list[index_of_parameter] = parameter_list[index_of_parameter] + 1
        return(best_list, index_of_parameter, evaluation, best_list)

In [1816]:
model_values = []
model_value = []

#THIS ONE IS DONE WITH THE SIGMOID BEFORE THE LEAKYRELU

#hp = [2, 1, 2, 3, 2, 3]
hp = [1,1,1,1,1,1]
pe = (1000, 1000)
bl = hp

for so_many_models in range(1000):
    
    for each_parameters in range(6): # for each parameter
    
        #return predicted model
        running = run_model(build_model(hp)) 

        #return evaluation of the model (2 values)
        eva = average_percentage_error(running) 

        print(hp, eva)

        #compare model with previous, return parameter list and index
        hp, each_parameters, pe, bl = good_parameter(eva, pe, hp, each_parameters, bl)

        # predicted = run_model(build_model(hp))

        # plt.title('first: f ='+str(hp[0])+'k ='+ str(hp[1]) + '\n second: f ='+ str(hp[2]) + 'k =' + str(hp[3]) +
        #  '\n pool size =' + str(hp[4]) + '\n Dense layer =' + str(hp[5]))
        # plt.scatter(test_labels['mutant_average'], test_labels['CRISPRoff_average'], c='black',s = 0.1)
        # plt.scatter(predicted[0], predicted[1], c='red',s = 0.1)
        # plt.figure()
        # plt.show()

        if abs(eva[0]) < 50 and abs(eva[1]) < 50:
            print('We fucking got a motherfucking good model bitch', eva, hp)


[1, 1, 1, 1, 1, 1] (64.22614452424135, -237.41724892290424)
[1, 2, 1, 1, 1, 1] (60.64750274623421, -245.3171381356713)
[1, 2, 2, 1, 1, 1] (405.1922642618783, 432.4582563950403)
[1, 2, 1, 2, 1, 1] (77.05839871811408, -224.4955554711991)
[1, 2, 1, 2, 2, 1] (19.86221879391772, -224.4041814285325)
[1, 2, 1, 2, 1, 2] (318.86443409784005, 14.833894841594951)
[2, 2, 1, 2, 1, 1] (27.83288400019977, -218.90666303906593)
[2, 3, 1, 2, 1, 1] (91.07822817933027, -236.1995557940928)
[2, 2, 2, 2, 1, 1] (1030.1156287669253, 1159.786533572236)
[2, 2, 1, 3, 1, 1] (77.51905565813108, -232.2806624464456)
[2, 2, 1, 3, 2, 1] (45.40877994749933, -237.95664829707195)
[2, 2, 1, 3, 2, 2] (175.4703429280843, -132.88822822165844)
[3, 2, 1, 3, 2, 1] (157.89961574552467, -155.16530728234795)
[3, 3, 1, 3, 2, 1] (150.86176656745224, -162.7207603041176)
[3, 3, 2, 3, 2, 1] (1753.7591985857143, 1884.7218743787057)
[3, 3, 1, 4, 2, 1] (131.73103138429298, -193.3596846793653)
[3, 3, 1, 4, 3, 1] (129.25506270111092, -197.98

KeyboardInterrupt: 